# Homework 4 - Kaggle Competition
## CSCI 5622 - Spring 2019
#Scott Scheraga
***
train-features.csv contains the features of the examples / cases, separated by commas, that you will use to train your classifiers. You must associate the features found in each row / line with the output found in the corresponding row / line in train-output.csv to create a full case.

test-features.csv contains the features of the examples / cases that you will need to predict using your trained model.

See train.names for header descriptions of the train and test features. This will be helpful in understanding how you may need to normalize, bin, or otherwise update your data to better suit your classifiers.

You can use test-submission.demo to see what an example submission file should look like. This file is a baseline case - you can even upload it to make sure your Kaggle account is working correctly!


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns

import pandas as pd
from sklearn import svm, metrics
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,roc_curve, roc_auc_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import RFECV
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split

from collections import defaultdict
import math
from random import seed
from random import random

%matplotlib inline



Batching notes:
Intial analysis of the training set showed the following information:
from print(traincombined['output'].value_counts())
        print(traincombined.groupby('output').mean())
24720 people below threshold, 7841 above.


As marital status averages were the following, I binned by married vs unmarried due to notably different outputs. 

--marital-status        |||         output                   
 Divorced                    0.104209  
 Married-AF-spouse           0.434783  
 Married-civ-spouse          0.446848  
 Married-spouse-absent       0.081340  
 Never-married               0.045961  
 Separated                   0.064390  
 Widowed                     0.085599  
 
Similarly, for relationship, I binned by Spouse vs not spouse for the same reason as above, and then decided that the relationship column was redunant to marital status, and then deleted it. 

--Relationship      |||  output                    
 Husband              0.448571  
 Not-in-family        0.103070  
 Other-relative       0.037717  
 Own-child            0.013220  
 Unmarried            0.063262  
 Wife                 0.475128 
 
Workclass:
workclass      |||output                                          
 Self-emp-inc      0.557348
 Federal-gov       0.386458
 Local-gov         0.294792
 Self-emp-not-inc  0.284927
 State-gov         0.271957
 Private           0.218673
 ?                 0.104031
 Without-pay       0.000000
 Never-worked      0.000000
 
There is a very notable increase in capital gain and loss for people above the income threshold. I decided on a 4 bins for capital gain and capital loss. 

--Averages for 0--
 age 36.783738 
 education-num 9.595065 
 capital-gain 148.752468
 capital-loss 53.142921 
 hours-per-week 38.840210  

--Averages for 1--
 age 44.249841
 education-num 11.611657  
 capital-gain 4006.142456  
 capital-loss 195.001530 
 hours-per-week 45.473026 




In [2]:
class Data:
    def __init__(self,features):
        self.features=features
        
    def preprocess(self):    
        
        #self.features= pd.read_csv("traincombined.csv")
        #self.features =self.features.dropna()
        #https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8
        self.features=self.features.drop(columns =
                 ["race",'native-country','education'])
        
        #self.features=self.features.drop(columns = 
          #  ["workclass", "educatio'DataFrame' object has no attribute 'ravel'n","marital-status","occupation",
         #    "relationship","race","relationship","occupation","sex","fnlwgt","native-country"])

        
        #print(self.features['education'].value_counts())
        self.features['sex']=np.where(self.features['sex'] ==' Female',
                                                1, self.features['sex'])
        self.features['sex']=np.where(self.features['sex'] ==' Male',
                                                0, self.features['sex'])
        
        #I might be losing data heere. Try splitting!
        """
        self.features['education']=np.where(self.features['education'] ==' Preschool',
                                                ' Some-school', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' 1st-4th',
                                                ' Some-school', self.features['education'])       
        self.features['education']=np.where(self.features['education'] ==' 5th-6th',
                                                ' Some-school', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' 7th-8th',
                                                ' Some-school', self.features['education'])  
        
        self.features['education']=np.where(self.features['education'] ==' 9th',
                                                ' Some-school', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' 10th',
                                                ' Some-school', self.features['education'])       
        self.features['education']=np.where(self.features['education'] ==' 11th',
                                                ' Some-school', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' 12th',
                                                ' Some-school', self.features['education'])
        
        
        self.features['education']=np.where(self.features['education'] ==' Assoc-voc',
                                                ' Assoc-professional', self.features['education'])    
        self.features['education']=np.where(self.features['education'] ==' Prof-school',
                                                ' Assoc-professional', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' Assoc-acdm',
                                                ' Assoc-professional', self.features['education'])        
        self.features['education']=np.where(self.features['education'] ==' Some-college',
                                                ' HS-grad or Some-college', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' HS-grad',
                                                ' HS-grad or Some-college', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' Masters',
                                                ' Graduate-degree', self.features['education'])
        self.features['education']=np.where(self.features['education'] ==' Doctorate',
                                                ' Graduate-degree', self.features['education'])
       
       
        self.features['marital-status']=np.where(self.features['marital-status'] ==' Divorced',
                                                ' Not Married', self.features['marital-status'])    
        self.features['marital-status']=np.where(self.features['marital-status'] ==' Never-married',
                                                ' Not Married', self.features['marital-status'])    
        self.features['marital-status']=np.where(self.features['marital-status'] ==' Separated',
                                                ' Not Married', self.features['marital-status'])    
        self.features['marital-status']=np.where(self.features['marital-status'] ==' Widowed',
                                                ' Not Married', self.features['marital-status'])  
        self.features['marital-status']=np.where(self.features['marital-status'] ==' Married-spouse-absent',
                                                ' Not Married', self.features['marital-status'])        
        self.features['marital-status']=np.where(self.features['marital-status'] ==' Married-civ-spouse',
                                                ' Married', self.features['marital-status'])    
        self.features['marital-status']=np.where(self.features['marital-status'] ==' Married-AF-spouse',
                                                ' Married', self.features['marital-status'])  
        
        self.features['workclass']=np.where(self.features['workclass'] ==' Never-worked',
                                                ' Zero', self.features['workclass']) 
        self.features['workclass']=np.where(self.features['workclass'] ==' Without-pay',
                                                ' Zero', self.features['workclass'])  
        self.features['workclass']=np.where(self.features['workclass'] ==' ?',
                                                ' Low', self.features['workclass'])  
        self.features['workclass']=np.where(self.features['workclass'] ==' Private',
                                                ' Low', self.features['workclass'])  
        self.features['workclass']=np.where(self.features['workclass'] ==' State-gov',
                                                ' Mid', self.features['workclass'])  
        self.features['workclass']=np.where(self.features['workclass'] ==' Self-emp-not-inc',
                                                ' Mid', self.features['workclass'])  
        self.features['workclass']=np.where(self.features['workclass'] ==' Local-gov',
                                                ' Mid', self.features['workclass'])  
        self.features['workclass']=np.where(self.features['workclass'] ==' Federal-gov',
                                                ' High', self.features['workclass'])  
        self.features['workclass']=np.where(self.features['workclass'] ==' Self-emp-inc',
                                                ' High', self.features['workclass'])  
        
        #Label Encoding
        
        self.features['workclass']=np.where(self.features['workclass'] ==' Never-worked',
                                                1, self.features['workclass']) 
        self.features['workclass']=np.where(self.features['workclass'] ==' Without-pay',
                                                1, self.features['workclass'])  
        self.features['workclass']=np.where(self.features['workclass'] ==' ?',
                                                2, self.features['workclass'])  
        self.features['workclass']=np.where(self.features['workclass'] ==' Private',
                                                2, self.features['workclass'])  
        self.features['workclass']=np.where(self.features['workclass'] ==' State-gov',
                                                3, self.features['workclass'])  
        self.features['workclass']=np.where(self.features['workclass'] ==' Self-emp-not-inc',
                                                3, self.features['workclass'])  
        self.features['workclass']=np.where(self.features['workclass'] ==' Local-gov',
                                                3, self.features['workclass'])  
        self.features['workclass']=np.where(self.features['workclass'] ==' Federal-gov',
                                                4, self.features['workclass'])  
        self.features['workclass']=np.where(self.features['workclass'] ==' Self-emp-inc',
                                                4, self.features['workclass']) 
        
        """
        
        #A lot of people dumped coutnry for good or bad...
        #0 mean output from test set
        country1 = [' Holand-Netherlands', ' Outlying-US(Guam-USVI-etc)']
        #less than 0.1 mean output from test set
        country2 = [' Dominican-Republic',' Columbia',' Guatemala',' Mexico', 
                    ' Nicaragua',' Peru',' Vietnam', ' Honduras',' Guatemala',' El-Salvador',' Haiti']
        # 0.1 to 0.2 mean output from test set
        country3 = [' Puerto-Rico',' Trinadad&Tobago',' Portugal',' Laos', 
                    ' Jamaica',' Ecuador',' Thailand']
        #0.2-0.3 mean output from test set
        country4 = [' Poland',' South',' Ireland',' Hungary',' United-States',
                    ' Scotland',' ?',' Ecuador',' Cuba',' China',' Greece'] 
        #0.3-0.4 mean output from test set
        country5 = [' Hong',' Philippines',' Germany',' Canada',' England',
                    ' Italy',' Cambodia',' Yugoslavia',' Japan',' Taiwan']         
        #greater than 0.4 mean output from test set
        country6 = [' India',' France',' Iran']      
        
        """  
        for x in country1:
            self.features['native-country']=np.where(self.features['native-country'] == x,
                                                ' countrygroup1', self.features['native-country']) 
        for x in country2:
            self.features['native-country']=np.where(self.features['native-country'] == x,
                                                ' countrygroup2', self.features['native-country']) 
        for x in country3:
            self.features['native-country']=np.where(self.features['native-country'] == x,
                                                ' countrygroup3', self.features['native-country'])         
        for x in country4:
            self.features['native-country']=np.where(self.features['native-country'] == x,
                                                ' countrygroup4', self.features['native-country']) 
        for x in country5:
            self.features['native-country']=np.where(self.features['native-country'] == x,
                                                ' countrygroup5', self.features['native-country'])         
        for x in country6:
            self.features['native-country']=np.where(self.features['native-country'] == x,
                                                ' countrygroup6', self.features['native-country'])          
        
        #Label Encoding
        
        for x in country1:
            self.features['native-country']=np.where(self.features['native-country'] == x,
                                                1, self.features['native-country']) 
        for x in country2:
            self.features['native-country']=np.where(self.features['native-country'] == x,
                                                2, self.features['native-country']) 
        for x in country3:
            self.features['native-country']=np.where(self.features['native-country'] == x,
                                                3, self.features['native-country'])         
        for x in country4:
            self.features['native-country']=np.where(self.features['native-country'] == x,
                                                4, self.features['native-country']) 
        for x in country5:
            self.features['native-country']=np.where(self.features['native-country'] == x,
                                                5, self.features['native-country'])         
        for x in country6:
            self.features['native-country']=np.where(self.features['native-country'] == x,
                                                6, self.features['native-country'])          
             
        """
        
        #https://towardsdatascience.com/preprocessing-with-sklearn-a-complete-and-comprehensive-guide-670cb98fcfb9
        #https://scikit-learn.org/stable/modules/preprocessing.html
        #Normalization of continuous data
        
        #self.features['education-num'] = preprocessing.scale(self.features['education-num'])
        #self.features["fnlwgt"] = preprocessing.scale(self.features["fnlwgt"])
        
        
        """
        normscalefeatures=['capital-gain','capital-loss','education-num',"hours-per-week","fnlwgt","age"]
        for n in normscalefeatures:
           # self.features[n] = preprocessing.normalize(self.features[n])
            self.features[n] = preprocessing.scale(self.features[n])
        """

        
        #scaler1 = MinMaxScaler(feature_range = (0,1))
        #scaler1.fit_transform(self.features['capital-gain'].values.reshape(-1, 1))
        #scaler2 = MinMaxScaler(feature_range = (0,1))
        #scaler2.fit_transform(self.features['capital-loss'].values.reshape(-1, 1))
        #scaler3 = MinMaxScaler()
        #scaler3.fit_transform(self.features['age'].values.reshape(-1, 1))
        #scaler4 = MinMaxScaler(feature_range = (0,1))
        #scaler4.fit_transform(self.features['education-num'].values.reshape(-1, 1))
        #scaler5 = MinMaxScaler()
        #scaler5.fit_transform(self.features['fnlwgt'].values.reshape(-1, 1))

        
     
        #self.features['capital-gain']=np.where(self.features['capital-gain'].astype(float) < 149,
          #                                      ' Low', self.features['capital-gain'])
        
        #self.features['capital-gain']=np.where((self.features['capital-gain'].astype(float) > 149) &
        #         (self.features['capital-gain'].astype(float) <2077),' Mid-Low', self.features['capital-gain'])        
        #self.features['capital-gain']=np.where((self.features['capital-gain'] > 2077) &
           #      (self.features['capital-gain'] <4006),' Mid-High', self.features['capital-gain']) 
       
        #self.features['capital-gain']=np.where(int(self.features['capital-gain']) > 4006,
        #                                        ' High', self.features['capital-gain'])        

        """
        capital losses: 0, 53.142921, 124.0722255, 195.001530, and up
        """

        #Reference: https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8

        
        category_variables=['marital-status','workclass',"occupation","relationship"] #'education',,'workclass'
        for var in category_variables:
            cat_list='var'+'_'+var
            cat_list = pd.get_dummies(self.features[var], prefix=var)
            data1=self.features.join(cat_list)
            self.features=data1
            
        category_variables=['marital-status','workclass',"occupation","relationship"] #,'education',"relationship"
        self.features_vars=self.features.columns.values.tolist()
        to_keep=[i for i in self.features_vars if i not in category_variables]
        
        
        
        data_final=self.features[to_keep]
        #data_final=data_final.drop(columns = [   
         #   'education_ Graduate-degree','education_ HS-grad or Some-college',
         #   'education_ Assoc-professional','education_ Bachelors']) 
        
        
        #'workclass_ Low','workclass_ Mid','education_ Bachelors'
        


        
        """
        data_final=data_final.drop(columns =
                 [,'workclass_ High'])
        
        data_final=data_final.drop(columns =['age','education-num','capital-loss',
         'education_ Assoc-professional','education_ Some-school',
         'native-country_ countrygroup1','native-country_ countrygroup3',
         'native-country_ countrygroup4','native-country_ countrygroup5'])
        
        
        data_final.columns.values
        
        #print(data_final)
        
        """
        
        return data_final
        
        


In [11]:
traindatafeatures= pd.read_csv("train-features_scottannotated.csv")
trainoutput= pd.read_csv("train-output_scottannotated.csv")
testdatafeatures= pd.read_csv("test-features_scottannotated.csv")  

data_train = Data(traindatafeatures).preprocess()
data_test= Data(testdatafeatures).preprocess()
#Data_train= (32561, 45)
#Data_Test= (16281, 45)

combinedData = pd.concat([data_train,data_test])

normscalefeatures=['capital-gain','capital-loss','education-num',"hours-per-week","fnlwgt","age"]
for n in normscalefeatures:
        # self.features[n] = preprocessing.normalize(self.features[n])
        #data_train[n] = preprocessing.scale(data_train[n])
        #data_test[n] = preprocessing.scale(data_test[n])
        combinedData[n] = preprocessing.scale(combinedData[n])

dfs = np.split(combinedData, [32561], axis=0)
data_train =dfs[0]
data_test= dfs[1]
#print(dfs[0].shape)
#print(dfs[1].shape)        
        
#print(trainoutput)
trainoutputArray=np.asarray(trainoutput)
np.transpose(trainoutputArray)

#data_test.preprocess()

#print(trainoutputArray.ravel())

#data_final_vars=data_train.columns.values.tolist()
#print(data_train.columns.values)


model = LogisticRegression(max_iter=7600)
"""
rfe = RFE(model, 20)
rfe = rfe.fit(data_train, trainoutputArray.ravel())
print(rfe.support_)
print(data_train.columns.values)
print (data_train.shape)
print(rfe.ranking_)
data_train= rfe.transform(data_train)
data_test= rfe.transform(data_test)
#
#print(data_train)

print (data_train.shape)


model = SVC(gamma='auto')
#model = RFECV(mod, step=1, cv=20)
#model = model.fit(data_train, trainoutputArray.ravel())
      
"""
print(data_train.columns.values)





#https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
# Add noisy features to make the problem harder
random_state = np.random.RandomState(7)
n_samples, n_features = data_train.shape
n_samplestest, n_featurestest = data_test.shape
data_train = np.c_[data_train, random_state.randn(n_samples, 5)]
data_test = np.c_[data_test, random_state.randn(n_samplestest, 5)]

x_train, x_holdout, y_train, y_holdout = train_test_split(
             data_train, trainoutputArray.ravel(), test_size=0.10, random_state=120)

#SMOTE Oversampling
#https://towardsdatascience.com/building-a-logistic-regression-in-python-step-by-step-becd4d56c9c8
#https://towardsdatascience.com/logistic-regression-model-tuning-with-scikit-learn-part-1-425142e01af5
sm = SMOTE(random_state = 34)
x_train_res, y_train_res = sm.fit_sample(x_train, y_train.ravel()) #Resampled data

  
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
#https://towardsdatascience.com/cross-validation-a-beginners-guide-5b8ca04962cd


trainindexcounter=0
tempscore=0
score=0
kf = KFold(n_splits=10) # Define the split - into 2 folds 
#kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
for train_index, test_index in kf.split(x_train):
    x_trainfold, x_testfold = x_train_res[train_index], x_train_res[test_index]
    y_trainfold, y_testfold = y_train_res[train_index], y_train_res[test_index]
    model.fit(x_trainfold, y_trainfold)
    score = model.score(x_holdout, y_holdout)
    trainscore = model.score(x_trainfold, y_trainfold)
    print("Epoch #:", trainindexcounter, " Score: ",score, "  Train Score: ",trainscore) 
    trainindexcounter+=1
    
    if score<bestscore:
        break
    tempscore=score
    
score=bestscore    
#print(confusion_matrix(y_holdout, modeloutput.predict(x_holdout)))
finalpredictions=model.predict(x_holdout)
#predictions= model.predict(data_test)
#print(predictions)



tn, fp, fn, tp = confusion_matrix(y_holdout, finalpredictions).ravel()

print ("   ")
print("FINAL SCORE: ",score)
print("tn=",tn," fp=" ,fp," fn=", fn," tp=", tp)
print ("   ")
print("R2 score:",  r2_score(y_holdout,finalpredictions))
print("Mean squared error:", mean_squared_error(
              y_holdout,finalpredictions))

#variance calc
#https://www.bmc.com/blogs/mean-squared-error-r2-and-variance-in-regression-analysis/
er = []
g = 0
for i in range(len(y_holdout)):
    #print( "actual=", y_holdout[i], " observed=", finalpredictions[i])
    x = (y_holdout[i] - finalpredictions[i]) **2
    er.append(x)
    g = g + x
v = np.var(er)

print ("Variance", v)
print ("   ")





#https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/
ns_probs = [0 for _ in range(len(y_holdout))]
ns_auc = roc_auc_score(y_holdout, ns_probs)

lr_probs=model.predict_proba(x_holdout)
lr_probs = lr_probs[:, 1]
fpr, tpr,_=roc_curve(y_holdout,lr_probs) 
lr_auc= roc_auc_score( y_holdout,finalpredictions)



plt.figure()
##Adding the ROC
plt.plot(fpr, tpr, color='red', lw=2,label='ROC curve') #lw=2,
##Random FPR and TPR
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
##Title and label
plt.xlabel('FPR- False Positive')
plt.ylabel('TPR-True Positive')
plt.title('ROC curve')
plt.show()



print("AUC No Skill=",ns_auc)
print("AUC Logisitic=",lr_auc)
"""
"""
#https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6


#"""
predictions= model.predict(data_test)
print(predictions)
    
        # dictionary of lists  
dict = {'Category': predictions }  
   
df = pd.DataFrame(dict) 
  
        # saving the dataframe 
df.to_csv('ScottSubmission3.csv') 
#"""

#how well it can predict its own data vs holdouts 

#svm + nerual nets may be good for this assignm,ent

#naive bayes-  potentially not great..
#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
#https://towardsdatascience.com/logistic-regression-using-python-sklearn-numpy-mnist-handwriting-recognition-matplotlib-a6b31e2b166a
#https://towardsdatascience.com/logistic-regression-model-tuning-with-scikit-learn-part-1-425142e01af5

['age' 'fnlwgt' 'education-num' 'sex' 'capital-gain' 'capital-loss'
 'hours-per-week' 'marital-status_ Divorced'
 'marital-status_ Married-AF-spouse' 'marital-status_ Married-civ-spouse'
 'marital-status_ Married-spouse-absent' 'marital-status_ Never-married'
 'marital-status_ Separated' 'marital-status_ Widowed' 'education_ 10th'
 'education_ 11th' 'education_ 12th' 'education_ 1st-4th'
 'education_ 5th-6th' 'education_ 7th-8th' 'education_ 9th'
 'education_ Assoc-acdm' 'education_ Assoc-voc' 'education_ Bachelors'
 'education_ Doctorate' 'education_ HS-grad' 'education_ Masters'
 'education_ Preschool' 'education_ Prof-school' 'education_ Some-college'
 'workclass_ ?' 'workclass_ Federal-gov' 'workclass_ Local-gov'
 'workclass_ Never-worked' 'workclass_ Private' 'workclass_ Self-emp-inc'
 'workclass_ Self-emp-not-inc' 'workclass_ State-gov'
 'workclass_ Without-pay' 'occupation_ ?' 'occupation_ Adm-clerical'
 'occupation_ Armed-Forces' 'occupation_ Craft-repair'
 'occupation_ Exec-ma

NameError: name 'bestscore' is not defined